In [1]:
from model.utils import WeightRepository

In [3]:
from model.sfcn_reg import RegressionSFCN

MIN_AGE = 3
MAX_AGE = 95
model = RegressionSFCN(prediction_range=(MIN_AGE, MAX_AGE))

ModuleNotFoundError: No module named 'ml'

In [ ]:
# Suppose you already have the weight path

WEIGHT_PATH = '/Users/kingpowa/Documents/Programming/.pyment/pyment/data/sfcn-regbrain-age-2022.h5'
torch_weights